<a href="https://colab.research.google.com/github/DANU011/practice-analysis/blob/main/coding_practice/2%EC%9C%A0%ED%98%95%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classifier

In [79]:
# 서비스 이탈예측 데이터
# 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
'''
x_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv
y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv
x_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv
데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형 ''';

import pandas as pd

# 데이터 로드
X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

display(X_train.head())
display(y_train.head())

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [80]:
print(X_train.info())
print(X_train.nunique())
print(X_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 609.4+ KB
None
CustomerId         6499
Surname            2289
CreditScore         459
Geography             3
Gender                4
Age                  69
Tenure               11
Balance     

In [81]:
# 데이터 전처리
drop_columns = ['CustomerId', 'Surname']
X_train_drop = X_train.drop(columns = drop_columns)
X_test_drop = X_test.drop(columns = drop_columns)

In [82]:
import sklearn
print(sklearn.__all__)
import sklearn.model_selection
print(dir(sklearn.model_selection))

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']
['BaseCrossValidator', 'BaseShuffleSplit', 'GridSearchCV', 'GroupKFold', 'GroupShuffleSplit', 'KFold', 'LearningCurveDisplay', 'LeaveOneGroupOut', 'LeaveOneOut', 'LeavePGroupsOut', 'LeavePOut', 'ParameterGrid', 'ParameterSampler', 'PredefinedSplit', 'RandomizedSearchCV', 'RepeatedKFold', 'RepeatedStratifiedKFold', 'ShuffleSplit', 'StratifiedGroupKFold', 'StratifiedK

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 원핫 인코딩

# 데이터프레임의 범주형 변수를 원핫 인코딩. 각 범주형 변수는 새로운 이진 열로 변환.
X_train_dummies = pd.get_dummies(X_train_drop)
y = y_train['Exited'] # target 변수

# 테스트 데이터에 대해서도 동일한 작업을 수행.
# 훈련 데이터와 동일한 원핫 인코딩 구조를 갖기 위해 훈련 데이터와 같은 열 순서를 갖도록함.
X_test_dummies = pd.get_dummies(X_test_drop)
# 훈련 데이터와 테스트 데이터를 원핫 인코딩한 후에는 두 데이터프레임이 동일한 특성(열)을 가져야 함.
# 테스트 데이터의 열 순서를 훈련 데이터와 동일하게 맞춰주는 과정.
# 만약 테스트 데이터에 훈련 데이터에 없는 새로운 범주가 있으면 해당 열은 버려짐.
X_test_dummies = X_test_dummies[X_train_dummies.columns]

# print(help(train_test_split))
X_train, X_validation, y_train, y_validation = train_test_split(X_train_dummies, y, test_size = 0.33, random_state = 42)
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=23)

In [84]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [85]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# model_score 훈련 및 검증 세트에 대한 예측 및 양성 클래스의 예측 확률을 생성

# 훈련 데이터에 대한 예측을 생성
# rf.predict 메서드는 입력 데이터에 대한 모델의 예측된 클래스(여기서는 이탈 여부)를 반환.
predict_train_label = rf.predict(X_train)

# 훈련 데이터에 대한 양성 클래스(이탈)의 예측 확률을 생성
# rf.predict_proba 메서드는 각 클래스에 대한 확률을 반환 [:, 1]는 양성 클래스의 확률만을 선택.
predict_train_proba = rf.predict_proba(X_train)[:,1]

# 검증 데이터에 대한 예측을 생성
predict_validation_label = rf.predict(X_validation)
# 검증 데이터에 대한 양성 클래스의 예측 확률을 생성
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

In [86]:
# 문제에서 묻는 것에 따라 모델 성능 확인
# 정확도 (accuracy) , f1_score , recall , precision -> model.predict
# auc , 확률이라는 표현 -> model.predict_proba로 결과 출력 첫번째 행의 값을 가져오기 model.predict_proba()[:,1]

# 모델 지표 출력
print('train accuracy :', accuracy_score(y_train,predict_train_label))
print('validation accuracy :', accuracy_score(y_validation,predict_validation_label))
print('\n')
print('train f1_score :', f1_score(y_train,predict_train_label))
print('validation accuracy :', f1_score(y_validation,predict_validation_label))
print('\n')
print('train recall_score :', recall_score(y_train,predict_train_label))
print('validation recall_score :', recall_score(y_validation,predict_validation_label))
print('\n')
print('train precision_score :', precision_score(y_train,predict_train_label))
print('validation precision_score :', precision_score(y_validation,predict_validation_label))
print('\n')
print('train auc :', roc_auc_score(y_train,predict_train_proba))
print('validation auc :', roc_auc_score(y_validation,predict_validation_prob))

# test 데이터에 대한 모델 예측 마찬가지로 위와 같은 방식
predict_test_label = rf.predict(X_test_dummies)
predict_test_proba = rf.predict_proba(X_test_dummies)[:,1]

# csv로 예측 저장
#
# accuracy, f1_score, recall, precision
#pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_label}).to_csv('003000000.csv', index=False)

# auc, 확률
#pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_proba}).to_csv('003000000.csv', index=False)

train accuracy : 1.0
validation accuracy : 0.8652680652680653


train f1_score : 1.0
validation accuracy : 0.5912305516265912


train recall_score : 1.0
validation recall_score : 0.4543478260869565


train precision_score : 1.0
validation precision_score : 0.8461538461538461


train auc : 1.0
validation auc : 0.8497613211198555


In [87]:
# 이직여부 판단 데이터
# 데이터 설명 : 이직여부 판단 데이터 (target: 1: 이직 , 0 : 이직 x)
'''x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists (참고, 데이터 수정)''';

import pandas as pd
X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv')

display(X_train.head())
display(y_train.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0


In [88]:
print(X_train.info())
print('\n')
print(X_train.nunique())
print('\n')
print(X_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.2+ MB
None


enrollee_id               

In [89]:
# 범주형 변수인데 많은 unique값을 가진 컬럼은 날림.
drop_columns = ['enrollee_id', 'city', 'city_development_index', 'experience', 'training_hours']
X_train_drop = X_train.drop(columns = drop_columns)
X_test_drop = X_test.drop(columns = drop_columns)

In [90]:
import sklearn
import sklearn.model_selection
print(sklearn.__all__)
print(dir(sklearn.model_selection))

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']
['BaseCrossValidator', 'BaseShuffleSplit', 'GridSearchCV', 'GroupKFold', 'GroupShuffleSplit', 'KFold', 'LearningCurveDisplay', 'LeaveOneGroupOut', 'LeaveOneOut', 'LeavePGroupsOut', 'LeavePOut', 'ParameterGrid', 'ParameterSampler', 'PredefinedSplit', 'RandomizedSearchCV', 'RepeatedKFold', 'RepeatedStratifiedKFold', 'ShuffleSplit', 'StratifiedGroupKFold', 'StratifiedK

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train_dummies = pd.get_dummies(X_train_drop)
y = y_train['target'].astype('int')

X_test_dummies = pd.get_dummies(X_test_drop)
# train과 컬럼 순서 맞추기
X_test_dummies = X_test_dummies[X_train_dummies.columns]

# RandomForest 모델 훈련
X_train, X_validation, y_train, y_validation = train_test_split(X_train_dummies, y, test_size = 0.33, random_state = 42)
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=23)

In [92]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [93]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# model_score
predict_train_label = rf.predict(X_train)
predcit_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_proba = rf.predict_proba(X_validation)[:,1]

# 모델 성능 확인
print('train acc :', accuracy_score(y_train, predict_train_label))
print('val acc :', accuracy_score(y_validation, predict_validation_label))
print('\n')
print('train f1 :', f1_score(y_train, predict_train_label))
print('val f1 :', f1_score(y_validation, predict_validation_label))
print('\n')
print('train pre :', precision_score(y_train, predict_train_label))
print('val pre :', precision_score(y_validation, predict_validation_label))
print('\n')
print('train rec :', recall_score(y_train, predict_train_label))
print('val rec :', recall_score(y_validation, predict_validation_label))
print('\n')
print('train roc :', roc_auc_score(y_train, predcit_train_proba))
print('val roc :', roc_auc_score(y_validation, predict_validation_proba))

train acc : 0.8334931671062096
val acc : 0.7255474452554744


train f1 : 0.5862377122430742
val f1 : 0.32777115613825986


train pre : 0.7610208816705336
val pre : 0.4317111459968603


train rec : 0.47674418604651164
val rec : 0.2641690682036503


train roc : 0.8762173756146109
val roc : 0.6376763826796363


In [94]:
# test data
predict_test_label = rf.predict(X_test_dummies)
predict_test_proba = rf.predict_proba(X_test_dummies)[:,1]

# accuracy, f1_score, recall, precision
pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': predict_test_label}).to_csv('003000000.csv', index=False)

# auc, 확률
pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': predict_test_proba}).to_csv('003000000.csv', index=False)

In [95]:
# 정시 배송 여부 판단
#데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

''' x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/prachi13/customer-analytics (참고, 데이터 수정)''';

import pandas as pd
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')

display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [96]:
print(x_train.info())
print(x_train.nunique())
print(x_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB
None
ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Pro

In [97]:
# 범주형 변수인데 많은 unique값을 가진 컬럼 삭제.
drop_columns = ['ID']
x_train_drop = x_train.drop(columns = drop_columns)
x_test_drop = x_test.drop(columns = drop_columns)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 더미화
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['Reached.on.Time_Y.N']

x_test_dummies = pd.get_dummies(x_test_drop)
# 컬럼 순서 맞추기
x_test_dimmies = x_test_dummies[x_train_dummies.columns]

# 데이터 분할
x_train, x_validation, y_train, y_validation = train_test_split(x_train_dummies, y, test_size = 0.33, random_state = 42)

# 모델 학습
rf = RandomForestClassifier(random_state = 23)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=23)

In [98]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [103]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# model_score
predict_train_label = rf.predict(x_train)
predict_train_proba = rf.predict_proba(x_train)[:, 1]

predict_validation_label = rf.predict(x_validation)
predict_validation_proba = rf.predict_proba(x_validation)[:, 1]

# 결과 확인
print('train accuracy_score :', accuracy_score(y_train, predict_train_label))
print('val accuracy_score :', accuracy_score(y_validation, predict_validation_label))
print('\n')
print('train f1_score :', f1_score(y_train, predict_train_label))
print('val f1_score :', f1_score(y_validation, predict_validation_label))
print('\n')
print('train precision_score :', precision_score(y_train, predict_train_label))
print('val precision_score :', precision_score(y_validation, predict_validation_label))
print('\n')
print('train recall_score :', recall_score(y_train, predict_train_label))
print('val recall_score :', recall_score(y_validation, predict_validation_label))
print('\n')
print('train roc_auc_score :', roc_auc_score(y_train, predict_train_proba))
print('val roc_auc_score :', roc_auc_score(y_validation, predict_validation_proba))

train accuracy_score : 1.0
val accuracy_score : 0.6395775941230487


train f1_score : 1.0
val f1_score : 0.6744089589382


train precision_score : 1.0
val precision_score : 0.7245989304812834


train recall_score : 1.0
val recall_score : 0.630721489526765


train roc_auc_score : 1.0
val roc_auc_score : 0.7261997118475008


In [104]:
# test data
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]